# Alz specific gene analysis
A recent research on Alzheimer disease at 2019 identified around 200 significant genes via GWAS and by GWAS significant SNPs. 
This analysis take these regions, cross reference them with the genes that were sequenced in the ROSMAP data set, and then compute the weight and Association stats for it.

## Region list construction.
Before this step I take the 29 genes from table 1 and 192 genes from table S-13 and combined them into one csv file. It turns out that there are 210 unique genes combined. The genes that are presented in both tables are

"ADAMTS4"    "HESX1"      "TREM2"      "CNTNAP2"    "MS4A6A"     "PICALM"     "SLC24A4"    "ADAM10"     "SCIMP"      "ALPK2"     "APOE"       "AC074212.3" "CD33"       "CASS4"

Since for the first 29 genes the paper only report the gene name instead of ensemble ID which is how the genes in the ROSMAP data set labled. The annotation of extracted gene list is the first step.

After the gene are fully labled with the external ID. The number of genes that are included in the DLPFC gene expression  of ROSMAP study was retrieved.

It truns out that only 144 out of the 210 genes are sequenced in the ROSMAP project. Potentially due to some of them not on a somatic chromosome.


In [ ]:
library("biomaRt")
library(dplyr)
gene_list = readr::read_csv("/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Gene_list_from_alzhemer.csv")
## Getting the overlaped genes
gene_list[1:192,]$Name[gene_list[1:192,]$Name%in%gene_list[193:nrow(gene_list),]$Name]

##  gene ensemble ID annotation
ensembl = useDataset("hsapiens_gene_ensembl",mart=useMart("ENSEMBL_MART_ENSEMBL"))
ensembl_df <- getBM(attributes=c("ensembl_gene_id","chromosome_name", "start_position", "end_position","external_gene_name"),mart=ensembl)
annotated_genes = ensembl_df%>%filter(external_gene_name %in% gene_list$Name )%>%dplyr::select(ensembl_gene_id,external_gene_name)

## Crossref with Rosmap gene list.
RSM_gene_list = readr::read_delim("/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/geneTpmResidualsAgeGenderAdj_rename_region_list.txt","\t")
RSM_gene_list%>%filter(gene_ID%in%gene_list_anno$ensembl_gene_id)%>%readr::write_delim("/Users/haosun/Documents/WG_Reasearch_Assisstant/Rosmap/Data/RSM_Alz_region_list.txt",delim = "\t")


## Weight Computation
The weight of each fo the extracted region are then computed using the standard TWAS/Fusion workflow. All five models are used. There are 43 genes who passed the heritability check. The computation of weight for this 43 genes take around 9 hours. The genes that passed the heratibiity check can be found at `/home/hs3163/Project/Alz/data/Alz_passed_gene_region_list.txt` 
The Computed genes can be found at
`/home/hs3163/Project/Alz/WEIGHTS/`

In [ ]:
nohup sos run ~/GIT/neuro-twas/Workflow/twas_fusion.ipynb compute_wgt  \
  --molecular-pheno  /home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt  \
  --wd /home/hs3163/Project/Alz \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/Alz/data/Alz_passed_gene_region_list.txt \
  --region_name 1 \
  --data_start 2 \
  --output_path /home/hs3163/Project/Alz \
  --window 500000 \
  --container /home/hs3163/system_file/twas_latest.sif \
  --model bslmm blup lasso top1 enet \
  --job_size 1\
  -J 6 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

## Distribution of heritability
The distribution of heritability is as followed. This section contains the Explorational data analysis regarding the heritability of the 144 genes. More detailed will be added. 